In [16]:
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
import requests
from bs4 import BeautifulSoup as bs

# 검색부
def load_Korean_game(gametitle, display=5, pageno=1):
    url = f'https://www.grac.or.kr/WebService/GameSearchSvc.asmx/game?display={display}&pageno={pageno}&gametitle={gametitle}'
    
    response = requests.get(url)
    response.raise_for_status()  # 오류 발생 시 예외 처리

    # HTML 파싱 및 본문 내용 추출
    soup = bs(response.text, 'html.parser')
    content = soup.find_all('item')  # item 태그들 추출
    
    paragraphs = []
    for idx, i in enumerate(content):
        print(f'idx : {idx}')
        print('-'*50)
        print(f'content : {i}')
        title = i.find('gametitle')  # gametitle 태그 추출
        if title:
            print(f'gametitle : {title.text}')
            paragraphs.append(title.text)
    
    # 텍스트 변환 및 Document 객체로 변환
    text = "\n".join(paragraphs)
    documents = [Document(page_content=text, metadata={"source": url})]
    
    # 텍스트 스플릿
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    splits = text_splitter.split_documents(documents)
    
    return splits

load_Korean_game('화이트데이')


idx : 0
--------------------------------------------------
content : <item>
<rateno>GC-CC-NP-230203-016</rateno>
<rateddate>2023-02-03</rateddate>
<gametitle>화이트데이2: 거짓말하는 꽃(White Day2: The Flower That Tells Lies) (PC)</gametitle>
<orgname>게임콘텐츠등급분류위원회</orgname>
<entname>주식회사 루트엔스튜디오</entname>
<summary>화이트데이 전날 밤 사고로 폐쇄된 학교에서 숨겨진 진실을 찾는 공포 어드벤처 게임</summary>
<givenrate>15세이용가</givenrate>
<genre>어드벤처</genre>
<platform>PC/온라인 게임</platform>
<descriptors>공포</descriptors>
<cancelstatus>False</cancelstatus>
<canceleddate></canceleddate>
</item>
gametitle : 화이트데이2: 거짓말하는 꽃(White Day2: The Flower That Tells Lies) (PC)
idx : 1
--------------------------------------------------
content : <item>
<rateno>GC-CC-NP-180126-010</rateno>
<rateddate>2018-01-26</rateddate>
<gametitle>화이트데이: 담력시험</gametitle>
<orgname>게임콘텐츠등급분류위원회</orgname>
<entname>주식회사 손노리</entname>
<summary>학교의 폐쇠된 창고에서 진행하는 담력시험 게임으로 지시된 아이템들을 제한시간 내에 찾아오는 VR기기 전용 PC용 공포액션게임</summary>
<givenrate>12세이용가</givenrate>
<genre>액션</genre>
<pla

[Document(metadata={'source': 'https://www.grac.or.kr/WebService/GameSearchSvc.asmx/game?display=5&pageno=1&gametitle=화이트데이'}, page_content='화이트데이2: 거짓말하는 꽃(White Day2: The Flower That Tells Lies) (PC)\n화이트데이: 담력시험\n화이트데이: 학교라는 이름의 미궁\n화이트데이 : 학교라는 이름의 미궁\nWhite Day Triple (화이트데이트리플)')]

In [ ]:
display = 1000 # 한번에 출력하는 게임물 건 수 (최대:1000) // 필수
pageno = 1 # 검색 페이지 번호. 검색결과 페이지 당 display건의 데이터가 전송되며 display건 이상의 데이터가 존재할 경우 해당 페이지 번호를 전송해야 함. 없을 경우에는 1로 전송함 // 필수
gametitle = '화이트데이' # 검색하고자 하는 게임물명 // 선택

url = f'https://www.grac.or.kr/WebService/GameSearchSvc.asmx/game?display={display}&pageno={pageno}&gametitle={gametitle}'


# 임배딩
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

persist_directory = "./new_chroma_db"

vectorstore = Chroma.from_documents(docs, embedding=embedding_model, persist_directory="./chroma_db")